# Lab6 - WordNet

In [75]:
import requests as r
from functional import seq
from collections import namedtuple
from typing import Dict,List,Tuple
from fn import _
base ="http://api.slowosiec.clarin-pl.eu/plwordnet-api"

1. Read the [Wordnet API](http://api.slowosiec.clarin-pl.eu/docs/index.html)



2\. Get acquainted with [semantic relations](http://nlp.pwr.wroc.pl/narzedzia-i-zasoby/narzedzia/disaster/25-wiedza/81-relacje-w-slowosieci) in WordNet.

Nouns
 - hypernyms:
     - supersets of sth - animal is hypernym of dog
     - Y is a hypernym of X if every X is a (kind of) Y (canine is a hypernym of dog)
 - hyponyms:
     - subsets
     - Y is a hyponym of X if every Y is a (kind of) X (dog is a hyponym of canine)
 - coordinate terms:
     - siblings
     - Y is a coordinate term of X if X and Y share a hypernym (wolf is a coordinate term of dog, and dog is a coordinate term of wolf)
 - meronym:
     - being a part of
     - Y is a meronym of X if Y is a part of X (window is a meronym of building)
 - holonym:
     - being a whole thing (...)
     - Y is a holonym of X if X is a part of Y (building is a holonym of window)
     
     
Verbs
  - hypernym: 
      - supersets
      - the verb Y is a hypernym of the verb X if the activity X is a (kind of) Y (to perceive is an hypernym of to listen)
   - troponym:
       - doing sth else is to some degree (?)
       - the verb Y is a troponym of the verb X if the activity Y is doing X in some manner (to lisp is a troponym of to talk)
  - entailment:
        - the verb Y is entailed by X if by doing X you must be doing Y (to sleep is entailed by to snore)
  - coordinate terms:
      - siblings
      - those verbs sharing a common hypernym (to lisp and to yell)

3\. Find all meaning of the _szkoda_ **noun** and display all their synonyms.

In [107]:
def senses_search(word):
    return seq(r.get(base + "/senses/search",{"lemma":word}).json()['content'])

In [108]:
json = senses_search("szkoda")

In [109]:
json[0]

{'domain': {'description': 'plWN miejsca i umiejscowienie',
  'id': 12,
  'lexicon': {'id': 1,
   'language': 'pl',
   'lexiconIdentifier': 'P 2.2',
   'name': 'Słowosieć 2.2'},
  'name': 'plWN_msc'},
 'id': 8769,
 'lemma': {'id': 249489, 'word': 'szkoda'},
 'lexicon': {'id': 1,
  'language': 'pl',
  'lexiconIdentifier': 'P 2.2',
  'name': 'Słowosieć 2.2'},
 'partOfSpeech': {'id': 2,
  'lexicon': {'id': 1,
   'language': 'pl',
   'lexiconIdentifier': 'P 2.2',
   'name': 'Słowosieć 2.2'},
  'lmfType': 'noun',
  'name': 'plWN rzeczownik'},
 'senseNumber': 1}

In [65]:
def trim_sense(field:Dict):
    return {
        'lemma':field['lemma']['word'],
        'id':field['id'],
        "domain":field['domain']['description'],
        'partOfSpeech': field['partOfSpeech']['lmfType'],
        'sense' : field['senseNumber']
    }

In [66]:
entries = seq(json).map(trim_sense)
entries

szkoda,8769,plWN miejsca i umiejscowienie,noun,1
szkoda,8770,plWN czasowniki wyrażające uczucia,verb,1
szkoda,8771,plWN posiadanie i jego zmiana,noun,2


In [68]:
def synsetid_for_sense(senseid):
    return r.get(base+ "/senses/{}/synset".format(senseid)).json()['id']

def synset_by_id(synsetid):
    return r.get(base+ "/synsets/{}/senses".format(synsetid)).json()



def synonyms(senseid): 
    return seq(synset_by_id(synsetid_for_sense(senseid))).map(trim_sense)

In [55]:
synonyms(8769)

szkoda,8769,plWN miejsca i umiejscowienie,noun,1


In [56]:
synonyms(8770)

szkoda,8770,plWN czasowniki wyrażające uczucia,verb,1
żałować,11379,plWN czasowniki wyrażające uczucia,verb,3
żal,143137,plWN czasowniki wyrażające uczucia,verb,1


In [57]:
synonyms(8771)

strata,8495,plWN zdarzenia,noun,1
utrata,9582,plWN zdarzenia,noun,1
szkoda,8771,plWN posiadanie i jego zmiana,noun,2
uszczerbek,29762,plWN zdarzenia,noun,1


4\. Find closure of **hypernymy** relation for the first meaning of the _wypadek drogowy_ expression.
   Create diagram of the relations as a directed graph.
   

In [95]:
#find hypernyme relation id
Relation= namedtuple("Relation","id, name")
def relations():
    return seq(r.get(base+ "/relationTypes").json()).map(lambda entry: Relation(id=entry['id'],name= entry['name']))
relations()

id,name
10,hiponimia
11,hiperonimia
12,antonimia
13,konwersja
14,meronimia
15,holonimia
19,fuzzynimia
20,część
21,porcja
22,miejsce


In [79]:
hyperonymy_id = 11

In [138]:
def sense_relations(senseid):
    return r.get(base+ "/senses/{}/relations/type/11".format(senseid)).json()
#     return r.get(base+ "/senses/{}/relations/from".format(senseid)).json()
#     return r.get(base+ "/senses/{}/relations/to".format(senseid)).json()

In [136]:
senses_search("wypadek drogowy").map(trim_sense)

wypadek drogowy,644786,plWN zdarzenia,noun,1


In [137]:
sense_relations(644786)#.map(lambda entry: entry['relation']['id'])

[]

5\. Find direct **hyponyms** of _wypadek<sub>1</sub>_ noun.

6\. Find second-order **hyponyms** of the same noun.

7\. Display as a directed graph (with labels for the edges) semantic relations between the following groups of lexemes:
   1. szkoda<sub>2</sub>, strata<sub>1</sub>, uszczerbek<sub>1</sub>, szkoda majątkowa<sub>1</sub>, 
      uszczerbek na zdrowiu<sub>1</sub>, krzywda<sub>1</sub>, niesprawiedliwość<sub>1</sub>, nieszczęście<sub>2</sub>.
   1. wypadek<sub>1</sub>, wypadek komunikacyjny<sub>1</sub>, kolizja<sub>2</sub>, zderzenie<sub>2</sub>,
      kolizja drogowa<sub>1</sub>, bezkolizyjny<sub>2</sub>, katastrofa budowlana<sub>1</sub>, wypadek
      drogowy<sub>1</sub>.
      
8\. Find the value of [Leacock-Chodorow semantic similarity measure](ftp://www-vhost.cs.toronto.edu/public_html/public_html/pub/gh/Budanitsky+Hirst-2001.pdf)
   between following pairs of lexemes:
   1. szkoda<sub>2</sub> - wypadek<sub>1</sub>,
   1. kolizja<sub>2</sub> - szkoda majątkowa<sub>1</sub>,
   1. nieszczęście<sub>2</sub> - katastrofa budowlana<sub>1</sub>.